<a href="https://colab.research.google.com/github/SkyRanger2010/DE2024_PY/blob/main/HomeWork_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import os

data_path = 'src'

# Загрузка данных
contact_preference_src = pd.read_csv(os.path.join(data_path, 'contact_preference.csv'), sep=';')
customer_preferenced_contact_type_src = pd.read_csv(os.path.join(data_path, 'customer_preferenced_contact_type.csv'), sep=';')
customer_src = pd.read_csv(os.path.join(data_path, 'customer.csv'), sep=';')
customers_segments_src = pd.read_csv(os.path.join(data_path, 'customers_segmetns.csv'), sep=';')
location_src = pd.read_csv(os.path.join(data_path, 'location.csv'), sep=';')
product_feedback_src = pd.read_csv(os.path.join(data_path, 'product_feedback.csv'), sep=';')
product_src = pd.read_csv(os.path.join(data_path, 'product.csv'), sep=';')
purchase_src = pd.read_csv(os.path.join(data_path, 'purchase.csv'), sep=';')
segment_src = pd.read_csv(os.path.join(data_path, 'segment.csv'), sep=';')
service_feedback_src = pd.read_csv(os.path.join(data_path, 'service_feedback.csv'), sep=';')


In [ ]:
# Приведем данные к нужным типам
contact_preference=contact_preference_src.astype({'contact_id':'str'})
customer_preferenced_contact_type=customer_preferenced_contact_type_src.astype({'customer_id':'str','contact_type_id':'str'})
customer=customer_src.astype({'id':'str','location_id':'str'})
customers_segments=customers_segments_src.astype({'customer_id':'str', 'segment_id':'str'})
location=location_src.astype({'location_id':'str'})
product_feedback=product_feedback_src.astype({'id':'str', 'purchase_id':'int'})
product_feedback['purchase_id']=product_feedback['purchase_id'].round(0)
product_feedback['purchase_id']=product_feedback['purchase_id'].astype(str)
product=product_src.astype({'id':'str'})
product['price']=pd.to_numeric(product['price'],errors='coerce').fillna(0)
purchase=purchase_src.astype({'id':'str', 'user_id':'str','product_id':'str'})
segment=segment_src.astype({'id':'str'})
service_feedback=service_feedback_src.astype({'id':'str', 'customer_id':'str'})

In [ ]:
# Переименуем столбцы
customer.rename(columns={'id':'customer_id'}, inplace=True)
purchase.rename(columns={'user_id':'customer_id'}, inplace=True)
purchase.rename(columns={'id':'purchase_id'}, inplace=True)
product.rename(columns={'id':'product_id'}, inplace=True)
product_feedback.rename(columns={'id':'feedback_id'}, inplace=True)


In [ ]:
# Объединение таблиц
user_data = (
    customer.merge(customers_segments, on='customer_id', how='left', suffixes=('', '_segment'))
            .merge(purchase, on='customer_id', how='left', suffixes=('', '_purchase'))
            .merge(product, on='product_id', how='left', suffixes=('', '_product'))
            .merge(product_feedback, on='purchase_id', how='left', suffixes=('', '_feedback'))
)
user_data.sample(5)

,customer_id,name,email,location_id,registration_date,segment_id,purchase_id,product_id,purchase_date,quantity,product_name,category_name,price,feedback_id,rating,review
107,11,Kim,kim@example.com,1,2023-08-05,3,131071020,89,2023-09-15,1.0,Gears of War 5,Games,39.99,NaN,NaN,NaN
395,40,Nora,nora@example.com,10,2023-04-10,3,326855091,45,2023-05-25,1.0,Eleanor Oliphant Is Completely Fine,Books,12.99,197,7.0,"Decent quality, but expected more."
470,47,Uri,uri@example.com,7,2023-05-15,1,805524929,99,2023-06-21,2.0,Doom Eternal,Games,49.99,107,9.0,"Very good, will buy again."
14,2,Bob,bob@example.com,2,2023-02-20,3,852164781,131,2023-12-02,2.0,Zootopia,Movies,12.99,NaN,NaN,NaN
94,9,Ivy,ivy@example.com,9,2023-07-10,1,956349124,111,2023-06-04,2.0,Star Wars: A New Hope,Movies,14.99,NaN,NaN,NaN


In [ ]:
user_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501 entries, 0 to 500
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   customer_id        501 non-null    object 
 1   name               501 non-null    object 
 2   email              501 non-null    object 
 3   location_id        501 non-null    object 
 4   registration_date  501 non-null    object 
 5   segment_id         501 non-null    object 
 6   purchase_id        500 non-null    object 
 7   product_id         500 non-null    object 
 8   purchase_date      500 non-null    object 
 9   quantity           500 non-null    float64
 10  product_name       500 non-null    object 
 11  category_name      500 non-null    object 
 12  price              500 non-null    float64
 13  feedback_id        235 non-null    object 
 14  rating             235 non-null    float64
 15  review             235 non-null    object 
dtypes: float64(3), object(13)


In [ ]:
# Добавление колонки purchase_amount и обработка NaN датафрейма
user_data['purchase_amount'] = user_data['quantity'] * user_data['price']
user_data['quantity'] = user_data['quantity'].fillna(0)
user_data['price'] = user_data['price'].fillna(0)
user_data['rating'] = user_data['rating'].fillna(0)
user_data = user_data.fillna('-')

In [ ]:
user_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501 entries, 0 to 500
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   customer_id        501 non-null    object 
 1   name               501 non-null    object 
 2   email              501 non-null    object 
 3   location_id        501 non-null    object 
 4   registration_date  501 non-null    object 
 5   segment_id         501 non-null    object 
 6   purchase_id        501 non-null    object 
 7   product_id         501 non-null    object 
 8   purchase_date      501 non-null    object 
 9   quantity           501 non-null    float64
 10  product_name       501 non-null    object 
 11  category_name      501 non-null    object 
 12  price              501 non-null    float64
 13  feedback_id        501 non-null    object 
 14  rating             501 non-null    float64
 15  review             501 non-null    object 
 16  purchase_amount    501 non

In [ ]:
# Группировка витрины
user_summary = user_data.groupby('customer_id').agg(
    total_purchases=('purchase_id', 'count'),
    total_spent=('purchase_amount', 'sum'),
    average_rating=('rating', 'mean'),
    highest_rated_product=('rating', 'max'),
    most_popular_product_name=('product_name', pd.Series.mode) ,
    total_reviews=('feedback_id', 'count')
).reset_index()

In [ ]:
#Обработка данных витрины
user_summary['most_popular_product_name'] = user_summary['most_popular_product_name'].apply(lambda x: x[0] if type(x) != str else x) #выводим первый элемент списка самых популярных, если их несколько
user_summary['average_rating'] = user_summary['average_rating'].round(2) #округлим средний рейтинг до 2 знаков после запятой


In [ ]:
user_summary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 7 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   customer_id                50 non-null     object 
 1   total_purchases            50 non-null     int64  
 2   total_spent                50 non-null     float64
 3   average_rating             50 non-null     float64
 4   highest_rated_product      50 non-null     float64
 5   most_popular_product_name  50 non-null     object 
 6   total_reviews              50 non-null     int64  
dtypes: float64(3), int64(2), object(2)
memory usage: 2.9+ KB


In [ ]:
user_summary.sample(10)

,customer_id,total_purchases,total_spent,average_rating,highest_rated_product,most_popular_product_name,total_reviews
2,11,11,362.87,3.45,10.0,Splatoon 2,11
42,48,11,402.83,4.27,10.0,A Star Is Born,11
48,8,13,433.32,4.85,10.0,Assassin's Creed,13
26,33,3,85.95,9.67,10.0,Chess Set,3
34,40,15,519.76,2.73,10.0,Among Us,15
17,25,11,412.83,3.18,10.0,Black Panther,11
4,13,7,200.87,2.43,9.0,A Star Is Born,7
18,26,15,629.26,2.87,10.0,1984,15
21,29,8,232.90,2.38,7.0,The Bell Jar,8
1,10,10,267.85,2.40,10.0,Eleanor Oliphant Is Completely Fine,10


In [ ]:
def calculate_spending_share(user_id):
    # Фильтруем данные по пользователю
    user_purchases = purchase[purchase['customer_id'] == user_id]

    # Вычисляем общие траты
    purchase_by_user = user_purchases.merge(product, how='left', on='product_id')
    purchase_by_user['amount'] = purchase_by_user.apply(lambda x:  float(x['price']) * float(x['quantity']), axis=1)
    total_spent = purchase_by_user['amount'].sum()

    # Вычисляем траты по категориям
    category_spending = purchase_by_user[purchase_by_user['category_name'].isin(['Movies', 'Games', 'Books'])].groupby('category_name')['amount'].sum().reset_index()

    # Добавляем долю от общих трат
    category_spending['share'] = category_spending['amount'] / total_spent

    return category_spending



In [ ]:
# Пример вызова функции для конкретного пользователя
user_id_example = '43'  # Замените на нужный ID пользователя
spending_share = calculate_spending_share(user_id_example)
print(spending_share)

  category_name  amount     share
0         Books   97.41  0.652139
1         Games    0.00  0.000000
2        Movies   51.96  0.347861
